# Data Extraction

This notebook parses team data-files containing results of past Australian Football League (AFL) matches, and creates a single data file representing a combined, temporal graph of all matches across all (selected) seasons.

See the [introduction](1_introduction.ipynb#Background "Section: Background") for further background information about the AFL and Australian Rules football.

## Parse the Matches

The data-files were formed by saving the entire match data (over all seasons) separately for each team, found as web pages on [AFL Tables](https://afltables.com/afl/afl_index.html "afltables.com"). Note that it doesn't matter what file ending is used, e.g. `.htm` or `.html`. 

However, the filenames **must** match the team names, since the data-file for each team records only the opposing team names. In the case of teams that have changed name over time (excluding those that have merged with other teams), we name the data-file after the modern team name, and manually remap this to the older team name(s) when appropriate.

Also note that these data-files contain matches for both the AFL and its predecessor, the VFL (Victorian Football League). We extract only the AFL data from 1990 onwards.

In [1]:
import sys
import os

sys.path.append(os.path.join("..", "python"))

In [2]:
import match_lib

In [3]:
team_files = match_lib.get_team_files(os.path.join("..", "matches"))

In [4]:
matches = {}
for team_file in team_files:
    print("Parsing:", team_file)
    team_name = match_lib.parse_team_name(team_file)
    team_matches = match_lib.parse_team_seasons(team_file, min_season=1990)
    matches[team_name] = team_matches

Parsing: ../matches/Melbourne.htm
Parsing: ../matches/Brisbane Bears.htm
Parsing: ../matches/Fitzroy.htm
Parsing: ../matches/Geelong.htm
Parsing: ../matches/Port Adelaide.html
Parsing: ../matches/West Coast.html
Parsing: ../matches/North Melbourne.htm
Parsing: ../matches/Adelaide.html
Parsing: ../matches/Collingwood.htm
Parsing: ../matches/Essendon.htm
Parsing: ../matches/St Kilda.html
Parsing: ../matches/Gold Coast.htm
Parsing: ../matches/Hawthorn.htm
Parsing: ../matches/Sydney.html
Parsing: ../matches/Richmond.html
Parsing: ../matches/Carlton.htm
Parsing: ../matches/Western Bulldogs.html
Parsing: ../matches/Fremantle.htm
Parsing: ../matches/Brisbane Lions.htm
Parsing: ../matches/Greater Western Sydney.htm


In [5]:
def get_match_fields():
    for team_name, team_matches in matches.items():
        for season, df_matches in team_matches.items():
            return list(df_matches.columns)

In [6]:
print(get_match_fields())

['Rnd', 'T', 'Opponent', 'Scoring3', 'F', 'Scoring5', 'A', 'R', 'M', 'W-D-L', 'Venue', 'Crowd', 'Date']


Note the deduplication of the field name `Scoring` for the scores of both teams.

## Construct the Graph

For convenience, we may consider each team as a vertex in a graph, and each match as an
edge between vertices. In order that each match is represented exactly once, we arbitrarily designate one of the teams to be the *'for'* team, and the opposing team to be
the *'against'* team. Hence, each edge is directed from the 'for' team to the 'against' team, and the match outcome (i.e. win, draw or loss) is specified with respect to the
'for' team.

### Define the team ordering

Since the match data-files are indexed by team name, this means that each match appears
twice over, i.e. once each in the respective data-files of the opposing teams.
In order to prevent edge duplication in our match graph, we first stipulate a canonical ordering of the teams, i.e. $T_1\prec T_2\prec T_3\cdots$. Then, for each match
with some team A versus some team B, if $A\prec B$ then we designate A as the 'for' team and B as the 'against' team. Conversely, if $A\succ B$, then we designate
A as the 'against' team and B as the 'for' team.

Note that although the canonical team ordering is arbitrary, the resulting 'for' and 'against' designations are deterministic and consistent.

In [7]:
def get_team_names():
    teams = set(matches.keys())
    for team_matches in matches.values():
        for df_matches in team_matches.values():
            teams |= set(df_matches.Opponent)
    return teams

In [8]:
known_teams = sorted(get_team_names())

In [9]:
print(known_teams)

['Adelaide', 'Brisbane Bears', 'Brisbane Lions', 'Carlton', 'Collingwood', 'Essendon', 'Fitzroy', 'Footscray', 'Fremantle', 'Geelong', 'Gold Coast', 'Greater Western Sydney', 'Hawthorn', 'Kangaroos', 'Melbourne', 'North Melbourne', 'Port Adelaide', 'Richmond', 'St Kilda', 'Sydney', 'West Coast', 'Western Bulldogs']


In [10]:
def precedes(team1, team2):
    return team1 < team2

### Define the edge attributes

For each match, we know environmental information such as the season, the match round within the season, the venue (i.e. the oval), and the date/time of each match.

For each team in the match, we also know if the venue is the team's home ground, and we know the numbers of goals and behinds scored in each quarter. We label each of the four quarters of a match by an integer suffix.

In [11]:
env_fields = ['season', 'round', 'datetime', 'venue']
quarter_score_fields = [f + str(i) for i in range(1, 5) for f in ['goals', 'behinds']]
team_fields = (
    ['team', 'is_home'] + quarter_score_fields + ['total_score', 'match_points']
)
for_team_fields = ['for_' + f for f in team_fields]
against_team_fields = ['against_' + f for f in team_fields]
result_fields = ['edge_type']
edge_fields = env_fields + for_team_fields + against_team_fields + result_fields

In [12]:
print(edge_fields)

['season', 'round', 'datetime', 'venue', 'for_team', 'for_is_home', 'for_goals1', 'for_behinds1', 'for_goals2', 'for_behinds2', 'for_goals3', 'for_behinds3', 'for_goals4', 'for_behinds4', 'for_total_score', 'for_match_points', 'against_team', 'against_is_home', 'against_goals1', 'against_behinds1', 'against_goals2', 'against_behinds2', 'against_goals3', 'against_behinds3', 'against_goals4', 'against_behinds4', 'against_total_score', 'against_match_points', 'edge_type']


Note that, for the time being, there is no vertex information defined, other than the team name.

### Process the matches

Note that we have to deal with the issue of teams changing names over time, due to the way the data are recorded. In particular, we have a data-file of matches for each team, but that team's name does not appear in the data-file, only the names of the
opposing teams.

Thus, for example, the `North Melbourne.htm` data-file contains matches that were played under the current name of "North Melbourne" and also under the old name of "Kangaroos". However, since neither of these names are explicitly listed in the data-file, we have to guess the correct team name from the filename and the season.

In [13]:
import pandas as pd

In [14]:
df_edges = pd.DataFrame(columns=edge_fields)

In [15]:
def split_score(score_str):
    goals, behinds = score_str.split('.')
    return int(goals), int(behinds)

def parse_quarter_scores(scores_str):
    scores = []
    prev_goals = 0
    prev_behinds = 0
    for score_str in scores_str.split(' '):
        goals, behinds = split_score(score_str)
        scores.append(goals - prev_goals)
        scores.append(behinds - prev_behinds)
        prev_goals = goals
        prev_behinds = behinds
    return scores

In [16]:
def correct_team_name(team, season):
    if team == "Western Bulldogs" and season <= 1996:
        return "Footscray"
    if team == "North Melbourne" and 1999 <= season <= 2007:
        return "Kangaroos"
    return team

In [17]:
num_accepted = 0
num_rejected = 0
for supposed_for_team, team_matches in matches.items():
    for season, df_matches in team_matches.items():
        for_team = correct_team_name(supposed_for_team, season)
        for match in df_matches.itertuples():
            against_team = match.Opponent
            if not precedes(for_team, against_team):
                # Ignore this edge; it will be extracted for the opposing team.
                num_rejected += 1
                continue
            num_accepted += 1
            env_info = [season, match.Rnd, match.Date, match.Venue]
            for_match_points = (4 if match.R == 'W' else 2 if match.R == 'D' else 0)
            against_match_points = 4 - for_match_points
            for_match_scores = parse_quarter_scores(match.Scoring3)
            against_match_scores = parse_quarter_scores(match.Scoring5)
            for_info = (
                [for_team, match.T == 'H'] + for_match_scores 
                + [int(match.F), for_match_points]
            )
            against_info = (
                [against_team, match.T == 'A'] + against_match_scores 
                + [int(match.A), against_match_points]
            )
            edge_type = (
                'defeated' if match.R == 'W' 
                else 'drew-with' if match.R == 'D' 
                else 'lost-to'
            )
            result_info = [edge_type]
            edge_info = env_info + for_info + against_info + result_info
            df_edges.loc[len(df_edges), :] = edge_info
assert num_accepted == num_rejected

### Order the matches

Note that the matches have been extracted in an arbitrary order.
For convenience, we reorder the matches from the earliest to the latest.

In [18]:
from datetime import datetime

In [19]:
date_fn = lambda s: datetime.strptime(s, "%a %d-%b-%Y %I:%M %p")

df_edges['_datetime'] = df_edges['datetime'].apply(date_fn)
df_edges.sort_values('_datetime', ascending=True, inplace=True)
df_edges.drop('_datetime', axis=1, inplace=True)

In [20]:
df_edges

,season,round,datetime,venue,for_team,for_is_home,for_goals1,for_behinds1,for_goals2,for_behinds2,...,against_behinds1,against_goals2,against_behinds2,against_goals3,against_behinds3,against_goals4,against_behinds4,against_total_score,against_match_points,edge_type
1018,1990,R1,Sat 31-Mar-1990 2:10 PM,Waverley Park,Geelong,True,5,3,2,3,...,4,6,7,9,7,10,6,192,4,lost-to
4943,1990,R1,Sat 31-Mar-1990 2:10 PM,Princes Park,Carlton,True,6,5,4,4,...,1,1,5,6,3,6,5,104,4,lost-to
300,1990,R1,Sat 31-Mar-1990 2:10 PM,M.C.G.,Melbourne,False,6,2,4,1,...,5,3,4,4,4,3,4,89,0,defeated
3375,1990,R1,Sat 31-Mar-1990 2:10 PM,Windy Hill,Essendon,True,7,4,6,7,...,1,2,4,1,3,2,4,60,0,defeated
435,1990,R1,Sat 31-Mar-1990 7:40 PM,Carrara,Brisbane Bears,True,4,3,3,2,...,4,4,5,1,3,3,2,74,0,defeated
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5067,2022,R18,Sat 16-Jul-2022 5:30 PM,Perth Stadium,Fremantle,True,3,3,3,2,...,1,2,7,3,4,5,4,82,4,lost-to
4330,2022,R18,Sat 16-Jul-2022 7:25 PM,M.C.G.,Carlton,True,4,1,1,0,...,4,4,2,3,2,2,5,85,4,lost-to
3686,2022,R18,Sun 17-Jul-2022 1:10 PM,M.C.G.,Hawthorn,True,2,3,7,0,...,2,3,1,2,2,3,0,77,0,defeated
7,2022,R18,Sun 17-Jul-2022 2:50 PM,Traeger Park,Melbourne,True,0,4,5,3,...,1,3,2,2,3,3,3,69,0,defeated


## Perform Sanity Checking

### Check goals, behinds and points

Each goal is worth 6 points, and each 'behind' is worth 1 point.

In [21]:
for_goals = (
    df_edges.for_goals1 + df_edges.for_goals2 
    + df_edges.for_goals3 + df_edges.for_goals4
)
for_behinds = (
    df_edges.for_behinds1 + df_edges.for_behinds2 
    + df_edges.for_behinds3 + df_edges.for_behinds4
)
for_scores = 6 * for_goals + for_behinds

In [22]:
ind = for_scores == df_edges.for_total_score
assert sum(ind) == len(ind)

In [23]:
against_goals = (
    df_edges.against_goals1 + df_edges.against_goals2 
    + df_edges.against_goals3 + df_edges.against_goals4
)
against_behinds = (
    df_edges.against_behinds1 + df_edges.against_behinds2 
    + df_edges.against_behinds3 + df_edges.against_behinds4
)
against_scores = 6 * against_goals + against_behinds

In [24]:
ind = against_scores == df_edges.against_total_score
assert sum(ind) == len(ind)

### Check known venues

Occasionally a team changes its home ground to another oval. More frequently, ovals change names due to sponsorship deals. In order to assess any statistics relating to the
match ground, we require a geographical understanding of the names and locations of the various ovals.

In [25]:
df_venues = pd.read_csv(os.path.join("..", "data", "venues.csv"))

In [26]:
df_venues

,venue,from,to,latitude,longitude,ground,suburb,state,aliases
0,Adelaide Oval,2011,NaN,-34.915600,138.596100,Adelaide Oval,Adelaide,SA,NaN
1,Bellerive Oval,2012,2019.0,-42.877300,147.373500,Bellerive Oval,Bellerive,TAS,Blundstone Arena
2,Blacktown,2012,2012.0,-33.769444,150.859167,Blacktown International Sportspark Oval,Rooty Hill,NSW,Blacktown ISP Oval; Blacktown ISP
3,Bruce Stadium,1995,1995.0,-35.250000,149.102778,Canberra Stadium,Bruce,ACT,GIO Stadium Canberra;GIO Stadium;Bruce Stadium...
4,Carrara,1987,NaN,-28.006300,153.366900,Carrara Stadium,Gold Coast,QLD,NaN
5,Cazaly's Stadium,2011,NaN,-16.935800,145.749200,Cazaly's Stadium,Westcourt,QLD,NaN
6,Docklands,2000,NaN,-37.816500,144.947400,Docklands Stadium,Melbourne,VIC,Marvel Stadium;Etihad Stadium;Telstra Dome;Col...
7,Eureka Stadium,2017,2019.0,-37.538410,143.848030,Eureka Stadium,Wendouree,VIC,Mars Stadium;Northern Oval #1;AUSTAR Arena
8,Football Park,1991,2013.0,-34.880000,138.495556,Football Park,West Lakes,SA,AAMI Stadium
9,Gabba,1981,NaN,-27.485900,153.038100,Brisbane Cricket Ground,Brisbane,QLD,NaN


In [27]:
df = pd.merge(df_edges, df_venues, on='venue', how='left')

In [28]:
assert not any(df.latitude.isna())

## Save the Graph

In [29]:
df_edges.to_csv(os.path.join("..", "data", "matches.csv"), index=False)